In [ ]:
# Name: Rohith Ganesan
# Student ID: 20553375


# Dataset: Arabic Levantine Hate Speech dataset
# Machine Learning Models Implemented - SVM, Logestic Regression, K-Nearest Neighbous , Random Forest, Multilayer Perceptron.
# Deep Learning Models Implements - RNN, Bi-LSTM, BERT.
# Data Augmented for Deep learning Models under (33%, 50%, 75%) of the training Data.

# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Importing the required packages
import pandas as pd
import numpy as np
import re
import timeit
import warnings
import string
import seaborn as sn
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.probability import FreqDist

# Importing Sci-kit learn functions for evaluation and dataset analysis purpose
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tkinter import ttk
from tkinter import *
import time
import plotly.figure_factory as ff

# Loading the ML Models used to classify the data into hate, offensive and neither category
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
pip install --upgrade emoji

# Preprocessing

In [ ]:
# Function used to pre-process the dataset
import re


def preprocessing(text):
    text = re.sub(r'@\S+', '', text)  # Remove @mentions
    text = re.sub(r'http\S+', '', text)  # Remove URLs starting with http
    text = re.sub(r'www\S+', '', text)  # Remove URLs starting with www
    text = re.sub(r'#\S+', '', text)  # Remove hashtags
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r' +', ' ', text).strip()  # Remove extra spaces and strip text
    #text = remove_emoji(text)  # Remove emojis using the custom function
    return text


In [ ]:
# Loading the Data set used for Validation and Training
df_train_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_train.csv") # ,names=["tweets","class"])
df_dev_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_test.csv") #,names=["tweets","class"])

In [ ]:
# Printing first 'n' data from the loaded training data dataframe
n = 10
df_train_orig.head(n)

,Tweet,Class
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,normal
1,سورية بلد الحضارات تربطها بعلية او بحيوان,normal
2,اخي الحاج اذا شعرت انك محرجا من الانتقادات لتص...,normal
3,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,normal
4,هذا البطل الذي قاتل وجاذف بحياته لتحيا انت يا ...,abusive
5,ولعها وهيك بتكون الشقيقة قدمت اكبر خدمة لشقيقت...,normal
6,قبل ما تترحم وسام الحسن كان مشارك بقتل الحريري...,normal
7,كلنا معك معالي الوزير الملك جبران باسيل,normal
8,ولك مبارح أيها المريض النفسي كنت عم تدافع عن ا...,normal
9,انو غريب يوجعك راسك القواتجية عادة بيكونو بلا راس,hate


In [ ]:
# Printing first 'n' data from the loaded training data dataframe
n = 10
df_dev_orig.head(n)

,Tweet,Class
0,وحاج حكي مثاليات وامه ومابعرف شو ،اخر هم عندكم...,normal
1,قطر مرادف الارهاب,hate
2,ارمني يوناني يحمل جنسية المانية من مواليد سوري...,hate
3,يا حمار قمة يعني زبالة,abusive
4,ما حدا غير يلي خلفتك عاهرة يا منحط يا سافل,abusive
5,قال جبران باسيل انو أمير قطر كسر الحصار على قط...,normal
6,صدقت ياصوت لبنان الحر العابق بالنضال والوطنية ...,normal
7,متابعةً لزيارة الوزير جبران باسيل الى وارسو، ع...,normal
8,عودة المياه إلى مجاريرها بين سوريا و الإمارات,normal
9,هي بتنطبق عليك وزير الصدفه,normal


In [ ]:
# Checking whether any data with null entries are present or not
df_train_orig[df_train_orig["Tweet"].isna()]

,Tweet,Class


In [ ]:
# Printing first 'n' data from the loaded validation data dataframe
n=10
df_dev_orig.head(n)

,Tweet,Class
0,وحاج حكي مثاليات وامه ومابعرف شو ،اخر هم عندكم...,normal
1,قطر مرادف الارهاب,hate
2,ارمني يوناني يحمل جنسية المانية من مواليد سوري...,hate
3,يا حمار قمة يعني زبالة,abusive
4,ما حدا غير يلي خلفتك عاهرة يا منحط يا سافل,abusive
5,قال جبران باسيل انو أمير قطر كسر الحصار على قط...,normal
6,صدقت ياصوت لبنان الحر العابق بالنضال والوطنية ...,normal
7,متابعةً لزيارة الوزير جبران باسيل الى وارسو، ع...,normal
8,عودة المياه إلى مجاريرها بين سوريا و الإمارات,normal
9,هي بتنطبق عليك وزير الصدفه,normal


In [ ]:
# Checking whether any data with null entries are present or not
df_dev_orig[df_dev_orig["Tweet"].isna()]

,Tweet,Class


In [ ]:
# Null entries filled with unknown tags
df_train = df_train_orig.fillna("unknown")
df_dev = df_dev_orig.fillna("unknown")

In [ ]:
# Printing total amount of data with null entries
print(df_train.isna().sum(),"\n")
print(df_dev.isna().sum())

Tweet    0
Class    0
dtype: int64 

Tweet    0
Class    0
dtype: int64


In [ ]:
# No. of classes present for this Arabic dataset
n_classes = len(np.unique(df_train["Class"]))
print("Number of classes:",n_classes)

Number of classes: 3


In [ ]:
# Defining the label-encoder
le = LabelEncoder()

# Obtaining the training and validation datasets
X_train = [(preprocessing(i)) for i in df_train["Tweet"]]
X_val = [(preprocessing(i)) for i in df_dev["Tweet"]]

Y_train = le.fit_transform(np.array(df_train_orig["Class"]))
Y_val = le.transform(np.array(df_dev_orig["Class"]))

In [ ]:
# Printing the class names
le.classes_

array(['abusive', 'hate', 'normal'], dtype=object)

In [ ]:
# Converting train data to dataframe
df = pd.DataFrame(X_train)

# Converting the string type labels to corresponding numeric tags
df["Class"] = le.transform(df_train["Class"])
df.head(n)

,0,Class
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,2
1,سورية بلد الحضارات تربطها بعلية او بحيوان,2
2,اخي الحاج اذا شعرت انك محرجا من الانتقادات لتص...,2
3,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,2
4,هذا البطل الذي قاتل وجاذف بحياته لتحيا انت يا ...,0
5,ولعها وهيك بتكون الشقيقة قدمت اكبر خدمة لشقيقتها,2
6,قبل ما تترحم وسام الحسن كان مشارك بقتل الحريري...,2
7,كلنا معك معالي الوزير الملك جبران باسيل,2
8,ولك مبارح أيها المريض النفسي كنت عم تدافع عن ا...,2
9,انو غريب يوجعك راسك القواتجية عادة بيكونو بلا راس,1


In [ ]:
# TF-IDF relation computed for the training and validation dataset - char_wb, char and word analyzers
tfidf_vec = TfidfVectorizer(ngram_range=(1,5),analyzer="char_wb",max_features=2000)
tfidf_vec.fit(X_train,Y_train)
tfidf_vec_char = TfidfVectorizer(ngram_range=(1,5),analyzer="char",max_features=2000)
tfidf_vec_char.fit(X_train,Y_train)
tfidf_vec_word = TfidfVectorizer(ngram_range=(1,5),analyzer="word",max_features=2000)
tfidf_vec_word.fit(X_train,Y_train)

TfidfVectorizer(max_features=2000, ngram_range=(1, 5))

In [ ]:
# Converting the transformed values to array
X_train_charwb =(tfidf_vec.transform(X_train)).toarray()
X_test_charwb = (tfidf_vec.transform(X_val)).toarray()
X_train_char =(tfidf_vec_char.transform(X_train)).toarray()
X_test_char = (tfidf_vec_char.transform(X_val)).toarray()
X_train_word =(tfidf_vec_word.transform(X_train)).toarray()
X_test_word = (tfidf_vec_word.transform(X_val)).toarray()

# Machine learning

In [ ]:
# Function for Logistic Regression Model
def run_logreg(x_train, y_train, x_test, y_test):

    clf = LogisticRegression(penalty="none", random_state=0, solver='saga')
    clf.fit(x_train, y_train)
    predicted = clf.predict(x_test)
    return clf, predicted, y_test

# Function for Linear SVM Model
def run_linsvm(x_train, y_train, x_test, y_test):
    clf = LinearSVC()
    clf.fit(x_train, y_train)
    predicted = clf.predict(x_test)
    return clf, predicted, y_test

# Function for Multilayer Perceptron Model
def run_mlptron(x_train, y_train, x_test, y_test):
    clf = MLPClassifier()
    clf.fit(x_train, y_train)
    predicted = clf.predict(x_test)
    return clf, predicted, y_test

# Function for Random Forest Model
def run_rand_forest(x_train, y_train, x_test, y_test):
  clf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
  clf.fit(x_train, y_train)
  predicted = clf.predict(x_test)
  return clf, predicted, y_test

# Function for KNN Model
def run_KNN(x_train, y_train, x_test, y_test):
  clf = KNeighborsClassifier()
  clf.fit(x_train, y_train)
  predicted = clf.predict(x_test)
  return clf, predicted, y_test

In [ ]:
# Function used to obtain the evaluation metrics -> Confusion Matrix, Class wise report for precision, recall and F1-score
def show_results(predicted, y_test, labels=[0,1,2]):
    clsr = classification_report(y_test, predicted, output_dict=True)
    cm = confusion_matrix(y_test, predicted)

    cr_df = pd.DataFrame(clsr).transpose()
    print(cr_df)

    print( " ")

    fig = ff.create_annotated_heatmap(cm)
    fig.update_layout(title_text='Confusion Matrix')
    fig.show()

In [ ]:
# Testing for KNN
clf, pred, y_test = run_KNN(X_train_charwb, Y_train, X_test_charwb, Y_val)


show_results(pred, Y_val)

              precision    recall  f1-score      support
0              0.538272  0.630058  0.580559   346.000000
1              0.288660  0.297872  0.293194    94.000000
2              0.796407  0.728767  0.761087   730.000000
accuracy       0.664957  0.664957  0.664957     0.664957
macro avg      0.541113  0.552232  0.544947  1170.000000
weighted avg   0.679276  0.664957  0.670109  1170.000000
 


In [ ]:
# Testing for Logistic Regression
clf, pred, Y_val = run_logreg(X_train_charwb, Y_train, X_test_charwb, Y_val)
show_results(pred, Y_val)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1183: FutureWarning:

`penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.



              precision    recall  f1-score      support
0              0.645455  0.615607  0.630178   346.000000
1              0.425287  0.393617  0.408840    94.000000
2              0.824701  0.850685  0.837492   730.000000
accuracy       0.744444  0.744444  0.744444     0.744444
macro avg      0.631814  0.619970  0.625503  1170.000000
weighted avg   0.739604  0.744444  0.741745  1170.000000
 


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



In [ ]:
# Testing for Linear SVM
clf, pred, y_test = run_linsvm(X_train_charwb, Y_train, X_test_charwb, Y_val)
show_results(pred, Y_val)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_classes.py:32: FutureWarning:

The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.



              precision    recall  f1-score      support
0              0.706081  0.604046  0.651090   346.000000
1              0.562500  0.191489  0.285714    94.000000
2              0.798100  0.920548  0.854962   730.000000
accuracy       0.768376  0.768376  0.768376     0.768376
macro avg      0.688894  0.572028  0.597255  1170.000000
weighted avg   0.751959  0.768376  0.748937  1170.000000
 


In [ ]:
# Testing for Multilayer-Perceptron
clf, pred, y_test = run_mlptron(X_train_charwb, Y_train, X_test_charwb, Y_val)
show_results(pred, Y_val)

              precision    recall  f1-score      support
0              0.635569  0.630058  0.632801   346.000000
1              0.365854  0.319149  0.340909    94.000000
2              0.820134  0.836986  0.828475   730.000000
accuracy       0.734188  0.734188  0.734188     0.734188
macro avg      0.607185  0.595398  0.600728  1170.000000
weighted avg   0.729056  0.734188  0.731437  1170.000000
 


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



In [ ]:
# Testing for Random Forest Classifier
clf, pred, y_test = run_rand_forest(X_train_charwb, Y_train, X_test_charwb, Y_val)
show_results(pred, y_test)

              precision    recall  f1-score      support
0              0.642612  0.540462  0.587127   346.000000
1              0.615385  0.085106  0.149533    94.000000
2              0.756351  0.897260  0.820802   730.000000
accuracy       0.726496  0.726496  0.726496     0.726496
macro avg      0.671449  0.507610  0.519154  1170.000000
weighted avg   0.711390  0.726496  0.697767  1170.000000
 


# RNN & Bi-LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def create_rnn_model(input_dim, hidden_units=64, output_units=3):
    model = tf.keras.Sequential()
    model.add(layers.SimpleRNN(hidden_units, return_sequences=False, input_shape=(input_dim, 1)))
    model.add(layers.Dense(64, activation='relu'))  # Softmax for multiclass classification
    model.add(layers.Dense(output_units, activation='softmax'))  # Softmax for multiclass classification
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Adjusted BiLSTM Model without embedding layer
def create_bilstm_model(input_dim, hidden_units=64, output_units=3):
    model = tf.keras.Sequential()
    model.add(layers.Bidirectional(layers.LSTM(hidden_units, return_sequences=False, input_shape=(input_dim, 1))))
    model.add(layers.Dense(64, activation='relu'))  # Softmax for multiclass classification
    model.add(layers.Dense(output_units, activation='softmax'))  # Softmax for multiclass classification
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
X_train_reshaped = X_train_charwb.reshape((X_train_charwb.shape[0], X_train_charwb.shape[1], 1))
X_test_reshaped = X_test_charwb.reshape((X_test_charwb.shape[0], X_test_charwb.shape[1], 1))

from tensorflow.keras.utils import to_categorical

# y_train and y_test contain the class labels (integers representing classes)
y_train_onehot = to_categorical(Y_train, num_classes=3)
y_val_onehot = to_categorical(Y_val, num_classes=3)



In [ ]:
# RNN Model
rnn_model = create_rnn_model(input_dim=X_train_reshaped.shape[1], output_units=3)  # 3 classes
rnn_model.summary()

rnn_model.fit(X_train_reshaped, y_train_onehot, epochs=5, batch_size=32, validation_data=(X_test_reshaped, y_val_onehot))
pred = rnn_model.predict(X_test_reshaped)
pred_class = tf.argmax(pred, axis=1)  # This gives the predicted class for each instance
y_val_class = tf.argmax(y_val_onehot, axis=1)
show_results(pred_class, y_val_class)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 64)                  │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,579 (33.51 KB)

 Trainable params: 8,579 (33.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 18s 338ms/step - accuracy: 0.6098 - loss: 0.9174 - val_accuracy: 0.6239 - val_loss: 0.8592
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 12s 274ms/step - accuracy: 0.6169 - loss: 0.8570 - val_accuracy: 0.6239 - val_loss: 0.8605
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 12s 271ms/step - accuracy: 0.6075 - loss: 0.8677 - val_accuracy: 0.6239 - val_loss: 0.8629
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 12s 271ms/step - accuracy: 0.6217 - loss: 0.8588 - val_accuracy: 0.6239 - val_loss: 0.8607
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 12s 271ms/step - accuracy: 0.6398 - loss: 0.8373 - val_accuracy: 0.6239 - val_loss: 0.8620
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step
              precision    recall  f1-score      support
0              0.000000  0.000000  0.000000   346.000000
1              0.000000  0.000000  0.000000    94.000000
2              0.623932  1.000000  0.768421   730.000000
accuracy       0.623932  0.623932  0.623932     0.623932
macro avg      0.207977  0.333333  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Bi-LSTM Model

bilstm_model = create_bilstm_model(input_dim=X_train_reshaped.shape[1], output_units=3)  # 3 classes
bilstm_model.summary()

bilstm_model.fit(X_train_reshaped, y_train_onehot, epochs=5, batch_size=32, validation_data=(X_test_reshaped, y_val_onehot))
pred = bilstm_model.predict(X_test_reshaped)
pred_class = tf.argmax(pred, axis=1)  # This gives the predicted class for each instance
y_val_class = tf.argmax(y_val_onehot, axis=1)
show_results(pred_class, y_val_class)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 10s 151ms/step - accuracy: 0.5659 - loss: 0.9993 - val_accuracy: 0.6239 - val_loss: 0.8775
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 144ms/step - accuracy: 0.6384 - loss: 0.8435 - val_accuracy: 0.6239 - val_loss: 0.8568
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 145ms/step - accuracy: 0.6235 - loss: 0.8519 - val_accuracy: 0.6239 - val_loss: 0.8565
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 144ms/step - accuracy: 0.6343 - loss: 0.8550 - val_accuracy: 0.6239 - val_loss: 0.8656
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 148ms/step - accuracy: 0.6279 - loss: 0.8458 - val_accuracy: 0.6239 - val_loss: 0.8580
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step
              precision    recall  f1-score      support
0              0.000000  0.000000  0.000000   346.000000
1              0.000000  0.000000  0.000000    94.000000
2              0.623932  1.000000  0.768421   730.000000
accuracy       0.623932  0.623932  0.623932     0.623932
macro avg      0.207977  0.333333  0.25

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



# BERT


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

from tensorflow.keras.optimizers import Adam

# Compiling the model using the Adam optimizer


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors="tf")
test_encodings = tokenizer(X_val, padding=True, truncation=True, max_length=128, return_tensors="tf")

# Y_train = to_categorical(np.array(df_train_orig["Class"].values)) # Replace df['class'] with actual label column
# Y_val = to_categorical(np.array(df_dev_orig["Class"].values))



# # One-hot encode the labels
# num_classes = 3  # Replace with the actual number of classes in your dataset
# y_train_onehot = to_categorical(Y_train, num_classes=num_classes)
# y_test_onehot = to_categorical(Y_val, num_classes=num_classes)

# # Check the shape of your one-hot encoded labels
# print(f"Y_train_onehot shape: {y_train_onehot.shape}")
# print(f"Y_test_onehot shape: {y_test_onehot.shape}")



# 3. Load BERT model for multiclass classification
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=3)

import tensorflow as tf



# Use AdamW from transformers
from transformers import AdamWeightDecay
bert_model.compile(optimizer=AdamWeightDecay(learning_rate=5e-5),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

bert_model.fit(
    [train_encodings['input_ids'], train_encodings['attention_mask']],
    Y_train,
    validation_data=([test_encodings['input_ids'], test_encodings['attention_mask']], Y_val),
    epochs=10,
    batch_size=16
)


# Evaluate the model
predictions = bert_model.predict([test_encodings['input_ids'], test_encodings['attention_mask']])
predicted_labels = np.argmax(predictions.logits, axis=1)

# Compute accuracy
accuracy = accuracy_score(Y_val, predicted_labels)

# Compute classification report
report = classification_report(Y_val, predicted_labels, digits=4)

# Print Results
print(f"Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n{report}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
293/293 [==============================] - 165s 473ms/step - loss: 1.1463 - accuracy: 0.3182 - val_loss: 1.0986 - val_accuracy: 0.2957
Epoch 2/10
293/293 [==============================] - 133s 452ms/step - loss: 1.0986 - accuracy: 0.3176 - val_loss: 1.0986 - val_accuracy: 0.2957
Epoch 3/10
293/293 [==============================] - 134s 459ms/step - loss: 1.1034 - accuracy: 0.3266 - val_loss: 1.0986 - val_accuracy: 0.6239
Epoch 4/10
293/293 [==============================] - 133s 453ms/step - loss: 1.0986 - accuracy: 0.5866 - val_loss: 1.0986 - val_accuracy: 0.6239
Epoch 5/10
293/293 [==============================] - 134s 457ms/step - loss: 1.0986 - accuracy: 0.5905 - val_loss: 1.0986 - val_accuracy: 0.6239
Epoch 6/10
293/293 [==============================] - 134s 457ms/step - loss: 1.0986 - accuracy: 0.5866 - val_loss: 1.0986 - val_accuracy: 0.6239
Epoch 7/10
293/293 [==============================] - 134s 457ms/step - loss: 1.0986 - accuracy: 0.5870 - val_loss: 1.0986 -

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



# Data Augmentation


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
from transformers import TFBertForSequenceClassification, BertTokenizer

data_splits = [0.3, 0.5, 0.75, 1]  # Different test sizes (0.3, 0.2, 0.1 for train-test splits)

# Loop over different data scenarios
for train_size in data_splits:
    print("-" * 30)
    print(f"\nRunning for train size {train_size}")
    print("-" * 30)

    # Loading the dataset for validation and training
    df_train_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_train.csv")
    df_dev_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_test.csv")

    df_train_orig = df_train_orig.sample(frac=train_size, random_state=42)  # random_state ensures reproducibility

    # Defining the label-encoder
    le = LabelEncoder()

    # Obtaining the training and validation datasets
    X_train = [(preprocessing(i)) for i in df_train_orig["Tweet"]]
    X_val = [(preprocessing(i)) for i in df_dev_orig["Tweet"]]

    Y_train = le.fit_transform(np.array(df_train_orig["Class"]))
    Y_val = le.transform(np.array(df_dev_orig["Class"]))

    # TF-IDF relation computed for the training and validation dataset - char_wb, char, and word analyzers
    tfidf_vec = TfidfVectorizer(ngram_range=(1, 5), analyzer="char_wb", max_features=2000)
    tfidf_vec.fit(X_train, Y_train)

    tfidf_vec_char = TfidfVectorizer(ngram_range=(1, 5), analyzer="char", max_features=2000)
    tfidf_vec_char.fit(X_train, Y_train)

    tfidf_vec_word = TfidfVectorizer(ngram_range=(1, 5), analyzer="word", max_features=2000)
    tfidf_vec_word.fit(X_train, Y_train)

    # Converting the transformed values to array
    X_train_charwb = tfidf_vec.transform(X_train).toarray()
    X_test_charwb = tfidf_vec.transform(X_val).toarray()
    X_train_char = tfidf_vec_char.transform(X_train).toarray()
    X_test_char = tfidf_vec_char.transform(X_val).toarray()
    X_train_word = tfidf_vec_word.transform(X_train).toarray()
    X_test_word = tfidf_vec_word.transform(X_val).toarray()


    X_train_reshaped = X_train_charwb.reshape((X_train_charwb.shape[0], X_train_charwb.shape[1], 1))
    X_test_reshaped = X_test_charwb.reshape((X_test_charwb.shape[0], X_test_charwb.shape[1], 1))

    y_train_onehot = to_categorical(Y_train, num_classes=3)
    y_val_onehot = to_categorical(Y_val, num_classes=3)

    # Training RNN
    rnn_model = create_rnn_model(input_dim=X_train_reshaped.shape[1], output_units=3)  # 3 classes
    rnn_model.summary()

    rnn_model.fit(X_train_reshaped, y_train_onehot, epochs=5, batch_size=32, validation_data=(X_test_reshaped, y_val_onehot))
    pred = rnn_model.predict(X_test_reshaped)
    pred_class = tf.argmax(pred, axis=1)  # This gives the predicted class for each instance
    y_val_class = tf.argmax(y_val_onehot, axis=1)
    show_results(pred_class, y_val_class)

    # Training BiLSTM
    bilstm_model = create_bilstm_model(input_dim=X_train_reshaped.shape[1], output_units=3)  # 3 classes
    bilstm_model.summary()

    bilstm_model.fit(X_train_reshaped, y_train_onehot, epochs=5, batch_size=32, validation_data=(X_test_reshaped, y_val_onehot))
    pred = bilstm_model.predict(X_test_reshaped)
    pred_class = tf.argmax(pred, axis=1)  # This gives the predicted class for each instance
    y_val_class = tf.argmax(y_val_onehot, axis=1)
    show_results(pred_class, y_val_class)

    # Load BERT model
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

    train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors="tf")
    test_encodings = tokenizer(X_val, padding=True, truncation=True, max_length=128, return_tensors="tf")

    bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=3)



    # Use AdamW from transformers
    from transformers import AdamWeightDecay
    bert_model.compile(optimizer=AdamWeightDecay(learning_rate=1e-6),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

    bert_model.fit(
        [train_encodings['input_ids'], train_encodings['attention_mask']],
        Y_train,
        validation_data=([test_encodings['input_ids'], test_encodings['attention_mask']], Y_val),
        epochs=10,
        batch_size=16
    )

    predictions = bert_model.predict([test_encodings['input_ids'], test_encodings['attention_mask']])
    predicted_labels = np.argmax(predictions.logits, axis=1)

    # Compute accuracy
    accuracy = accuracy_score(Y_val, predicted_labels)

    # Compute classification report
    report = classification_report(Y_val, predicted_labels, digits=4)

    # Print Results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Classification Report:\n{report}")


------------------------------

Running for train size 0.3
------------------------------


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)             │ (None, 64)                  │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,579 (33.51 KB)

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 19s 118ms/step - accuracy: 0.5889 - loss: 0.9285 - val_accuracy: 0.6239 - val_loss: 0.8582
Epoch 2/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 17s 116ms/step - accuracy: 0.6248 - loss: 0.8572 - val_accuracy: 0.6239 - val_loss: 0.8592
Epoch 3/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 17s 116ms/step - accuracy: 0.6143 - loss: 0.8681 - val_accuracy: 0.6239 - val_loss: 0.8587
Epoch 4/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 17s 116ms/step - accuracy: 0.6189 - loss: 0.8631 - val_accuracy: 0.6239 - val_loss: 0.8569
Epoch 5/5
147/147 ━━━━━━━━━━━━━━━━━━━━ 17s 116ms/step - accuracy: 0.6159 - loss: 0.8643 - val_accuracy: 0.6239 - val_loss: 0.8651
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step
              precision    recall  f1-score      support
0              0.000000  0.000000  0.000000   346.000000
1              0.000000  0.000000  0.000000    94.000000
2              0.623932  1.000000  0.768421   730.000000
accuracy       0.623932  0.623932  0.623932     0.623932
macro avg      0.207977  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
293/293 [==============================] - 41s 82ms/step - loss: 2.4774 - accuracy: 0.3599 - val_loss: 1.0949 - val_accuracy: 0.5051
Epoch 2/10
293/293 [==============================] - 21s 72ms/step - loss: 1.1799 - accuracy: 0.3997 - val_loss: 1.0958 - val_accuracy: 0.5410
Epoch 3/10
293/293 [==============================] - 21s 72ms/step - loss: 1.1554 - accuracy: 0.3858 - val_loss: 1.0986 - val_accuracy: 0.3761
Epoch 4/10
293/293 [==============================] - 21s 72ms/step - loss: 1.1098 - accuracy: 0.3482 - val_loss: 1.0986 - val_accuracy: 0.3983
Epoch 5/10
293/293 [==============================] - 21s 72ms/step - loss: 1.1158 - accuracy: 0.3732 - val_loss: 1.0986 - val_accuracy: 0.4154
Epoch 6/10
293/293 [==============================] - 21s 72ms/step - loss: 1.1078 - accuracy: 0.3608 - val_loss: 1.0986 - val_accuracy: 0.3265
Epoch 7/10
293/293 [==============================] - 21s 72ms/step - loss: 1.1743 - accuracy: 0.3715 - val_loss: 1.0977 - val_accuracy:

# 30% Data Augmentation

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
from transformers import TFBertForSequenceClassification, BertTokenizer

data_splits = [0.3]

# Loop over different data scenarios
for train_size in data_splits:
    print("-" * 30)
    print(f"\nRunning for train size {train_size}")
    print("-" * 30)

    # Loading the dataset for validation and training
    df_train_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_train.csv")
    df_dev_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_test.csv")

    df_train_orig = df_train_orig.sample(frac=train_size, random_state=42)  # random_state ensures reproducibility

    # Defining the label-encoder
    le = LabelEncoder()

    # Obtaining the training and validation datasets
    X_train = [(preprocessing(i)) for i in df_train_orig["Tweet"]]
    X_val = [(preprocessing(i)) for i in df_dev_orig["Tweet"]]

    Y_train = le.fit_transform(np.array(df_train_orig["Class"]))
    Y_val = le.transform(np.array(df_dev_orig["Class"]))

    # TF-IDF relation computed for the training and validation dataset - char_wb, char, and word analyzers
    tfidf_vec = TfidfVectorizer(ngram_range=(1, 5), analyzer="char_wb", max_features=2000)
    tfidf_vec.fit(X_train, Y_train)

    tfidf_vec_char = TfidfVectorizer(ngram_range=(1, 5), analyzer="char", max_features=2000)
    tfidf_vec_char.fit(X_train, Y_train)

    tfidf_vec_word = TfidfVectorizer(ngram_range=(1, 5), analyzer="word", max_features=2000)
    tfidf_vec_word.fit(X_train, Y_train)

    # Converting the transformed values to array
    X_train_charwb = tfidf_vec.transform(X_train).toarray()
    X_test_charwb = tfidf_vec.transform(X_val).toarray()
    X_train_char = tfidf_vec_char.transform(X_train).toarray()
    X_test_char = tfidf_vec_char.transform(X_val).toarray()
    X_train_word = tfidf_vec_word.transform(X_train).toarray()
    X_test_word = tfidf_vec_word.transform(X_val).toarray()


    X_train_reshaped = X_train_charwb.reshape((X_train_charwb.shape[0], X_train_charwb.shape[1], 1))
    X_test_reshaped = X_test_charwb.reshape((X_test_charwb.shape[0], X_test_charwb.shape[1], 1))

    y_train_onehot = to_categorical(Y_train, num_classes=3)
    y_val_onehot = to_categorical(Y_val, num_classes=3)

    # Training RNN
    rnn_model = create_rnn_model(input_dim=X_train_reshaped.shape[1], output_units=3)  # 3 classes
    rnn_model.summary()

    rnn_model.fit(X_train_reshaped, y_train_onehot, epochs=5, batch_size=32, validation_data=(X_test_reshaped, y_val_onehot))
    pred = rnn_model.predict(X_test_reshaped)
    pred_class = tf.argmax(pred, axis=1)  # This gives the predicted class for each instance
    y_val_class = tf.argmax(y_val_onehot, axis=1)
    show_results(pred_class, y_val_class)

    # Training BiLSTM
    bilstm_model = create_bilstm_model(input_dim=X_train_reshaped.shape[1], output_units=3)  # 3 classes
    bilstm_model.summary()

    bilstm_model.fit(X_train_reshaped, y_train_onehot, epochs=5, batch_size=32, validation_data=(X_test_reshaped, y_val_onehot))
    pred = bilstm_model.predict(X_test_reshaped)
    pred_class = tf.argmax(pred, axis=1)  # This gives the predicted class for each instance
    y_val_class = tf.argmax(y_val_onehot, axis=1)
    show_results(pred_class, y_val_class)

    # Load BERT model
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

    train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors="tf")
    test_encodings = tokenizer(X_val, padding=True, truncation=True, max_length=128, return_tensors="tf")

    bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=3)



    # Use AdamW from transformers
    from transformers import AdamWeightDecay
    bert_model.compile(optimizer=AdamWeightDecay(learning_rate=1e-6),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

    bert_model.fit(
        [train_encodings['input_ids'], train_encodings['attention_mask']],
        Y_train,
        validation_data=([test_encodings['input_ids'], test_encodings['attention_mask']], Y_val),
        epochs=10,
        batch_size=16
    )

    predictions = bert_model.predict([test_encodings['input_ids'], test_encodings['attention_mask']])
    predicted_labels = np.argmax(predictions.logits, axis=1)

    # Compute accuracy
    accuracy = accuracy_score(Y_val, predicted_labels)

    # Compute classification report
    report = classification_report(Y_val, predicted_labels, digits=4)

    # Print Results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Classification Report:\n{report}")


------------------------------

Running for train size 0.3
------------------------------


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_5 (SimpleRNN)             │ (None, 64)                  │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,579 (33.51 KB)

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 9s 152ms/step - accuracy: 0.6200 - loss: 0.9840 - val_accuracy: 0.6239 - val_loss: 0.8590
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 145ms/step - accuracy: 0.6319 - loss: 0.8484 - val_accuracy: 0.6239 - val_loss: 0.8567
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 144ms/step - accuracy: 0.6069 - loss: 0.8857 - val_accuracy: 0.6239 - val_loss: 0.8573
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 145ms/step - accuracy: 0.6271 - loss: 0.8497 - val_accuracy: 0.6239 - val_loss: 0.8702
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 6s 146ms/step - accuracy: 0.6464 - loss: 0.8512 - val_accuracy: 0.6239 - val_loss: 0.8569
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step
              precision    recall  f1-score      support
0              0.000000  0.000000  0.000000   346.000000
1              0.000000  0.000000  0.000000    94.000000
2              0.623932  1.000000  0.768421   730.000000
accuracy       0.623932  0.623932  0.623932     0.623932
macro avg      0.207977  0.333333  0.256

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
88/88 [==============================] - 29s 124ms/step - loss: 6.9914 - accuracy: 0.2858 - val_loss: 4.6934 - val_accuracy: 0.6128
Epoch 2/10
88/88 [==============================] - 8s 86ms/step - loss: 4.6096 - accuracy: 0.5403 - val_loss: 4.5538 - val_accuracy: 0.5889
Epoch 3/10
88/88 [==============================] - 8s 87ms/step - loss: 4.4471 - accuracy: 0.5053 - val_loss: 4.2563 - val_accuracy: 0.5863
Epoch 4/10
88/88 [==============================] - 8s 87ms/step - loss: 3.4647 - accuracy: 0.4882 - val_loss: 1.3913 - val_accuracy: 0.6000
Epoch 5/10
88/88 [==============================] - 8s 86ms/step - loss: 2.1074 - accuracy: 0.4170 - val_loss: 1.2237 - val_accuracy: 0.5462
Epoch 6/10
88/88 [==============================] - 8s 86ms/step - loss: 1.8468 - accuracy: 0.3863 - val_loss: 1.1811 - val_accuracy: 0.4128
Epoch 7/10
88/88 [==============================] - 7s 85ms/step - loss: 1.5181 - accuracy: 0.3614 - val_loss: 1.1432 - val_accuracy: 0.2940
Epoch 8/10


# 50% Data Augmentation

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
from transformers import TFBertForSequenceClassification, BertTokenizer

data_splits = [0.5]

# Loop over different data scenarios
for train_size in data_splits:
    print("-" * 30)
    print(f"\nRunning for train size {train_size}")
    print("-" * 30)

    # Loading the dataset for validation and training
    df_train_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_train.csv")
    df_dev_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_test.csv")

    df_train_orig = df_train_orig.sample(frac=train_size, random_state=42)  # random_state ensures reproducibility

    # Defining the label-encoder
    le = LabelEncoder()

    # Obtaining the training and validation datasets
    X_train = [(preprocessing(i)) for i in df_train_orig["Tweet"]]
    X_val = [(preprocessing(i)) for i in df_dev_orig["Tweet"]]

    Y_train = le.fit_transform(np.array(df_train_orig["Class"]))
    Y_val = le.transform(np.array(df_dev_orig["Class"]))

    # TF-IDF relation computed for the training and validation dataset - char_wb, char, and word analyzers
    tfidf_vec = TfidfVectorizer(ngram_range=(1, 5), analyzer="char_wb", max_features=2000)
    tfidf_vec.fit(X_train, Y_train)

    tfidf_vec_char = TfidfVectorizer(ngram_range=(1, 5), analyzer="char", max_features=2000)
    tfidf_vec_char.fit(X_train, Y_train)

    tfidf_vec_word = TfidfVectorizer(ngram_range=(1, 5), analyzer="word", max_features=2000)
    tfidf_vec_word.fit(X_train, Y_train)

    # Converting the transformed values to array
    X_train_charwb = tfidf_vec.transform(X_train).toarray()
    X_test_charwb = tfidf_vec.transform(X_val).toarray()
    X_train_char = tfidf_vec_char.transform(X_train).toarray()
    X_test_char = tfidf_vec_char.transform(X_val).toarray()
    X_train_word = tfidf_vec_word.transform(X_train).toarray()
    X_test_word = tfidf_vec_word.transform(X_val).toarray()


    X_train_reshaped = X_train_charwb.reshape((X_train_charwb.shape[0], X_train_charwb.shape[1], 1))
    X_test_reshaped = X_test_charwb.reshape((X_test_charwb.shape[0], X_test_charwb.shape[1], 1))

    y_train_onehot = to_categorical(Y_train, num_classes=3)
    y_val_onehot = to_categorical(Y_val, num_classes=3)

    # Training RNN
    rnn_model = create_rnn_model(input_dim=X_train_reshaped.shape[1], output_units=3)  # 3 classes
    rnn_model.summary()

    rnn_model.fit(X_train_reshaped, y_train_onehot, epochs=5, batch_size=32, validation_data=(X_test_reshaped, y_val_onehot))
    pred = rnn_model.predict(X_test_reshaped)
    pred_class = tf.argmax(pred, axis=1)  # This gives the predicted class for each instance
    y_val_class = tf.argmax(y_val_onehot, axis=1)
    show_results(pred_class, y_val_class)

    # Training BiLSTM
    bilstm_model = create_bilstm_model(input_dim=X_train_reshaped.shape[1], output_units=3)  # 3 classes
    bilstm_model.summary()

    bilstm_model.fit(X_train_reshaped, y_train_onehot, epochs=5, batch_size=32, validation_data=(X_test_reshaped, y_val_onehot))
    pred = bilstm_model.predict(X_test_reshaped)
    pred_class = tf.argmax(pred, axis=1)  # This gives the predicted class for each instance
    y_val_class = tf.argmax(y_val_onehot, axis=1)
    show_results(pred_class, y_val_class)

    # Load BERT model
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

    train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors="tf")
    test_encodings = tokenizer(X_val, padding=True, truncation=True, max_length=128, return_tensors="tf")

    bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=3)



    # Use AdamW from transformers
    from transformers import AdamWeightDecay
    bert_model.compile(optimizer=AdamWeightDecay(learning_rate=1e-6),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

    bert_model.fit(
        [train_encodings['input_ids'], train_encodings['attention_mask']],
        Y_train,
        validation_data=([test_encodings['input_ids'], test_encodings['attention_mask']], Y_val),
        epochs=10,
        batch_size=16
    )

    predictions = bert_model.predict([test_encodings['input_ids'], test_encodings['attention_mask']])
    predicted_labels = np.argmax(predictions.logits, axis=1)

    # Compute accuracy
    accuracy = accuracy_score(Y_val, predicted_labels)

    # Compute classification report
    report = classification_report(Y_val, predicted_labels, digits=4)

    # Print Results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Classification Report:\n{report}")


------------------------------

Running for train size 0.5
------------------------------


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_6 (SimpleRNN)             │ (None, 64)                  │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,579 (33.51 KB)

...

Epoch 1/5
74/74 ━━━━━━━━━━━━━━━━━━━━ 12s 133ms/step - accuracy: 0.6284 - loss: 0.9554 - val_accuracy: 0.6239 - val_loss: 0.8597
Epoch 2/5
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 134ms/step - accuracy: 0.6141 - loss: 0.8502 - val_accuracy: 0.6239 - val_loss: 0.8649
Epoch 3/5
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 129ms/step - accuracy: 0.6270 - loss: 0.8574 - val_accuracy: 0.6239 - val_loss: 0.8568
Epoch 4/5
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 130ms/step - accuracy: 0.6332 - loss: 0.8440 - val_accuracy: 0.6239 - val_loss: 0.8573
Epoch 5/5
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 129ms/step - accuracy: 0.6386 - loss: 0.8291 - val_accuracy: 0.6239 - val_loss: 0.8568
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step
              precision    recall  f1-score      support
0              0.000000  0.000000  0.000000   346.000000
1              0.000000  0.000000  0.000000    94.000000
2              0.623932  1.000000  0.768421   730.000000
accuracy       0.623932  0.623932  0.623932     0.623932
macro avg      0.207977  0.333333  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
147/147 [==============================] - 32s 99ms/step - loss: 2.8252 - accuracy: 0.4517 - val_loss: 1.7732 - val_accuracy: 0.6205
Epoch 2/10
147/147 [==============================] - 11s 78ms/step - loss: 1.9153 - accuracy: 0.5257 - val_loss: 1.7981 - val_accuracy: 0.6256
Epoch 3/10
147/147 [==============================] - 12s 79ms/step - loss: 1.7705 - accuracy: 0.5710 - val_loss: 1.7787 - val_accuracy: 0.6248
Epoch 4/10
147/147 [==============================] - 12s 78ms/step - loss: 1.7690 - accuracy: 0.5885 - val_loss: 1.7766 - val_accuracy: 0.6231
Epoch 5/10
147/147 [==============================] - 11s 78ms/step - loss: 1.7319 - accuracy: 0.6044 - val_loss: 1.7829 - val_accuracy: 0.6282
Epoch 6/10
147/147 [==============================] - 11s 78ms/step - loss: 1.7351 - accuracy: 0.6001 - val_loss: 1.7899 - val_accuracy: 0.6282
Epoch 7/10
147/147 [==============================] - 11s 78ms/step - loss: 1.7305 - accuracy: 0.5945 - val_loss: 1.7650 - val_accuracy:

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



# 75% Data Augmentation

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
from transformers import TFBertForSequenceClassification, BertTokenizer

data_splits = [0.75]

# Loop over different data scenarios
for train_size in data_splits:
    print("-" * 30)
    print(f"\nRunning for train size {train_size}")
    print("-" * 30)

    # Loading the dataset for validation and training
    df_train_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_train.csv")
    df_dev_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_test.csv")

    df_train_orig = df_train_orig.sample(frac=train_size, random_state=42)  # random_state ensures reproducibility

    # Defining the label-encoder
    le = LabelEncoder()

    # Obtaining the training and validation datasets
    X_train = [(preprocessing(i)) for i in df_train_orig["Tweet"]]
    X_val = [(preprocessing(i)) for i in df_dev_orig["Tweet"]]

    Y_train = le.fit_transform(np.array(df_train_orig["Class"]))
    Y_val = le.transform(np.array(df_dev_orig["Class"]))

    # TF-IDF relation computed for the training and validation dataset - char_wb, char, and word analyzers
    tfidf_vec = TfidfVectorizer(ngram_range=(1, 5), analyzer="char_wb", max_features=2000)
    tfidf_vec.fit(X_train, Y_train)

    tfidf_vec_char = TfidfVectorizer(ngram_range=(1, 5), analyzer="char", max_features=2000)
    tfidf_vec_char.fit(X_train, Y_train)

    tfidf_vec_word = TfidfVectorizer(ngram_range=(1, 5), analyzer="word", max_features=2000)
    tfidf_vec_word.fit(X_train, Y_train)

    # Converting the transformed values to array
    X_train_charwb = tfidf_vec.transform(X_train).toarray()
    X_test_charwb = tfidf_vec.transform(X_val).toarray()
    X_train_char = tfidf_vec_char.transform(X_train).toarray()
    X_test_char = tfidf_vec_char.transform(X_val).toarray()
    X_train_word = tfidf_vec_word.transform(X_train).toarray()
    X_test_word = tfidf_vec_word.transform(X_val).toarray()


    X_train_reshaped = X_train_charwb.reshape((X_train_charwb.shape[0], X_train_charwb.shape[1], 1))
    X_test_reshaped = X_test_charwb.reshape((X_test_charwb.shape[0], X_test_charwb.shape[1], 1))

    y_train_onehot = to_categorical(Y_train, num_classes=3)
    y_val_onehot = to_categorical(Y_val, num_classes=3)



    # Load BERT model
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

    train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors="tf")
    test_encodings = tokenizer(X_val, padding=True, truncation=True, max_length=128, return_tensors="tf")

    bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=3)



    # Use AdamW from transformers
    from transformers import AdamWeightDecay
    bert_model.compile(optimizer=AdamWeightDecay(learning_rate=1e-6),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

    bert_model.fit(
        [train_encodings['input_ids'], train_encodings['attention_mask']],
        Y_train,
        validation_data=([test_encodings['input_ids'], test_encodings['attention_mask']], Y_val),
        epochs=5,
        batch_size=16
    )

    predictions = bert_model.predict([test_encodings['input_ids'], test_encodings['attention_mask']])
    predicted_labels = np.argmax(predictions.logits, axis=1)

    # Compute accuracy
    accuracy = accuracy_score(Y_val, predicted_labels)

    # Compute classification report
    report = classification_report(Y_val, predicted_labels, digits=4)

    # Print Results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Classification Report:\n{report}")


------------------------------

Running for train size 0.75
------------------------------


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
220/220 [==============================] - 35s 89ms/step - loss: 1.5188 - accuracy: 0.3781 - val_loss: 0.8766 - val_accuracy: 0.6231
Epoch 2/5
220/220 [==============================] - 17s 76ms/step - loss: 0.9351 - accuracy: 0.6079 - val_loss: 0.8472 - val_accuracy: 0.6239
Epoch 3/5
220/220 [==============================] - 17s 75ms/step - loss: 0.9359 - accuracy: 0.6256 - val_loss: 0.8855 - val_accuracy: 0.6239
Epoch 4/5
220/220 [==============================] - 17s 76ms/step - loss: 0.8687 - accuracy: 0.6296 - val_loss: 0.8152 - val_accuracy: 0.6239
Epoch 5/5
37/37 [==============================] - 4s 36ms/step
Accuracy: 0.6239
Classification Report:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       346
           1     0.0000    0.0000    0.0000        94
           2     0.6239    1.0000    0.7684       730

    accuracy                         0.6239      1170
   macro avg     0.2080    0.3333    0.2561      117

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
from transformers import TFBertForSequenceClassification, BertTokenizer

data_splits = [0.75]

# Loop over different data scenarios
for train_size in data_splits:
    print("-" * 30)
    print(f"\nRunning for train size {train_size}")
    print("-" * 30)

    # Loading the dataset for validation and training
    df_train_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_train.csv")
    df_dev_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_test.csv")

    df_train_orig = df_train_orig.sample(frac=train_size, random_state=42)  # 0.1 represents 10%, and random_state ensures reproducibility

    # Defining the label-encoder
    le = LabelEncoder()

    # Obtaining the training and validation datasets
    X_train = [(preprocessing(i)) for i in df_train_orig["Tweet"]]
    X_val = [(preprocessing(i)) for i in df_dev_orig["Tweet"]]

    Y_train = le.fit_transform(np.array(df_train_orig["Class"]))
    Y_val = le.transform(np.array(df_dev_orig["Class"]))

    # TF-IDF relation computed for the training and validation dataset - char_wb, char, and word analyzers
    tfidf_vec = TfidfVectorizer(ngram_range=(1, 5), analyzer="char_wb", max_features=2000)
    tfidf_vec.fit(X_train, Y_train)

    tfidf_vec_char = TfidfVectorizer(ngram_range=(1, 5), analyzer="char", max_features=2000)
    tfidf_vec_char.fit(X_train, Y_train)

    tfidf_vec_word = TfidfVectorizer(ngram_range=(1, 5), analyzer="word", max_features=2000)
    tfidf_vec_word.fit(X_train, Y_train)

    # Converting the transformed values to array
    X_train_charwb = tfidf_vec.transform(X_train).toarray()
    X_test_charwb = tfidf_vec.transform(X_val).toarray()
    X_train_char = tfidf_vec_char.transform(X_train).toarray()
    X_test_char = tfidf_vec_char.transform(X_val).toarray()
    X_train_word = tfidf_vec_word.transform(X_train).toarray()
    X_test_word = tfidf_vec_word.transform(X_val).toarray()


    X_train_reshaped = X_train_charwb.reshape((X_train_charwb.shape[0], X_train_charwb.shape[1], 1))
    X_test_reshaped = X_test_charwb.reshape((X_test_charwb.shape[0], X_test_charwb.shape[1], 1))

    y_train_onehot = to_categorical(Y_train, num_classes=3)  # Replace 3 with the number of classes
    y_val_onehot = to_categorical(Y_val, num_classes=3)    # Replace 3 with the number of classes



    # Load BERT model
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

    train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors="tf")
    test_encodings = tokenizer(X_val, padding=True, truncation=True, max_length=128, return_tensors="tf")

    bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=3)



    # Use AdamW from transformers
    from transformers import AdamWeightDecay
    bert_model.compile(optimizer=AdamWeightDecay(learning_rate=1e-6),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

    bert_model.fit(
        [train_encodings['input_ids'], train_encodings['attention_mask']],
        Y_train,
        validation_data=([test_encodings['input_ids'], test_encodings['attention_mask']], Y_val),
        epochs=5,
        batch_size=16
    )

    predictions = bert_model.predict([test_encodings['input_ids'], test_encodings['attention_mask']])
    predicted_labels = np.argmax(predictions.logits, axis=1)

    # Compute accuracy
    accuracy = accuracy_score(Y_val, predicted_labels)

    # Compute classification report
    report = classification_report(Y_val, predicted_labels, digits=4)

    # Print Results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Classification Report:\n{report}")


------------------------------

Running for train size 0.75
------------------------------


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
220/220 [==============================] - 38s 100ms/step - loss: 4.1124 - accuracy: 0.5680 - val_loss: 4.9165 - val_accuracy: 0.6222
Epoch 2/5
220/220 [==============================] - 17s 78ms/step - loss: 3.8155 - accuracy: 0.5791 - val_loss: 2.7173 - val_accuracy: 0.6231
Epoch 3/5
220/220 [==============================] - 17s 77ms/step - loss: 2.0596 - accuracy: 0.4725 - val_loss: 1.1551 - val_accuracy: 0.3248
Epoch 4/5
220/220 [==============================] - 17s 77ms/step - loss: 1.2017 - accuracy: 0.3547 - val_loss: 1.0986 - val_accuracy: 0.3436
Epoch 5/5
37/37 [==============================] - 5s 36ms/step
Accuracy: 0.3744
Classification Report:
              precision    recall  f1-score   support

           0     0.3198    0.8671    0.4673       346
           1     0.0857    0.0638    0.0732        94
           2     0.8148    0.1808    0.2960       730

    accuracy                         0.3744      1170
   macro avg     0.4068    0.3706    0.2788      11

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
from transformers import TFBertForSequenceClassification, BertTokenizer

data_splits = [0.5]

# Loop over different data scenarios
for train_size in data_splits:
    print("-" * 30)
    print(f"\nRunning for train size {train_size}")
    print("-" * 30)

    # Loading the dataset for validation and training
    df_train_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_train.csv")
    df_dev_orig = pd.read_csv("/content/drive/MyDrive/Disseration UON/Datasets/arabic_test.csv")

    df_train_orig = df_train_orig.sample(frac=train_size, random_state=42)  # 0.1 represents 10%, and random_state ensures reproducibility

    # Defining the label-encoder
    le = LabelEncoder()

    # Obtaining the training and validation datasets
    X_train = [(preprocessing(i)) for i in df_train_orig["Tweet"]]
    X_val = [(preprocessing(i)) for i in df_dev_orig["Tweet"]]

    Y_train = le.fit_transform(np.array(df_train_orig["Class"]))
    Y_val = le.transform(np.array(df_dev_orig["Class"]))

    # TF-IDF relation computed for the training and validation dataset - char_wb, char, and word analyzers
    tfidf_vec = TfidfVectorizer(ngram_range=(1, 5), analyzer="char_wb", max_features=2000)
    tfidf_vec.fit(X_train, Y_train)

    tfidf_vec_char = TfidfVectorizer(ngram_range=(1, 5), analyzer="char", max_features=2000)
    tfidf_vec_char.fit(X_train, Y_train)

    tfidf_vec_word = TfidfVectorizer(ngram_range=(1, 5), analyzer="word", max_features=2000)
    tfidf_vec_word.fit(X_train, Y_train)

    # Converting the transformed values to array
    X_train_charwb = tfidf_vec.transform(X_train).toarray()
    X_test_charwb = tfidf_vec.transform(X_val).toarray()
    X_train_char = tfidf_vec_char.transform(X_train).toarray()
    X_test_char = tfidf_vec_char.transform(X_val).toarray()
    X_train_word = tfidf_vec_word.transform(X_train).toarray()
    X_test_word = tfidf_vec_word.transform(X_val).toarray()


    X_train_reshaped = X_train_charwb.reshape((X_train_charwb.shape[0], X_train_charwb.shape[1], 1))
    X_test_reshaped = X_test_charwb.reshape((X_test_charwb.shape[0], X_test_charwb.shape[1], 1))

    y_train_onehot = to_categorical(Y_train, num_classes=3)  # Replace 3 with the number of classes
    y_val_onehot = to_categorical(Y_val, num_classes=3)    # Replace 3 with the number of classes



    # Load BERT model
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

    train_encodings = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors="tf")
    test_encodings = tokenizer(X_val, padding=True, truncation=True, max_length=128, return_tensors="tf")

    bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-multilingual-uncased', num_labels=3)



    # Use AdamW from transformers
    from transformers import AdamWeightDecay
    bert_model.compile(optimizer=AdamWeightDecay(learning_rate=1e-6),
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

    bert_model.fit(
        [train_encodings['input_ids'], train_encodings['attention_mask']],
        Y_train,
        validation_data=([test_encodings['input_ids'], test_encodings['attention_mask']], Y_val),
        epochs=5,
        batch_size=16
    )

    predictions = bert_model.predict([test_encodings['input_ids'], test_encodings['attention_mask']])
    predicted_labels = np.argmax(predictions.logits, axis=1)

    # Compute accuracy
    accuracy = accuracy_score(Y_val, predicted_labels)

    # Compute classification report
    report = classification_report(Y_val, predicted_labels, digits=4)

    # Print Results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Classification Report:\n{report}")


------------------------------

Running for train size 0.5
------------------------------


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
147/147 [==============================] - 36s 106ms/step - loss: 1.0955 - accuracy: 0.5205 - val_loss: 0.9319 - val_accuracy: 0.6342
Epoch 2/5
147/147 [==============================] - 12s 82ms/step - loss: 0.9988 - accuracy: 0.5958 - val_loss: 0.9031 - val_accuracy: 0.6368
Epoch 3/5
147/147 [==============================] - 12s 82ms/step - loss: 0.9396 - accuracy: 0.6228 - val_loss: 0.8715 - val_accuracy: 0.6350
Epoch 4/5
147/147 [==============================] - 12s 82ms/step - loss: 0.8941 - accuracy: 0.6146 - val_loss: 0.8514 - val_accuracy: 0.6265
Epoch 5/5
37/37 [==============================] - 4s 36ms/step
Accuracy: 0.6248
Classification Report:
              precision    recall  f1-score   support

           0     0.6667    0.0058    0.0115       346
           1     0.0000    0.0000    0.0000        94
           2     0.6247    0.9986    0.7686       730

    accuracy                         0.6248      1170
   macro avg     0.4304    0.3348    0.2600      11

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

